In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [16]:
# Goal of the project 

# input an image from the test set
# The model will output predicted labels (e.g., [1, 8, 19]).
# These labels will then be mapped to a caption using:
 #- A pre-defined lookup table for captions associated with labels.
 #- or a text generation model 

train = '/kaggle/input/multi-label-classification-competition-2024/COMP5329S1A2Dataset/train.csv'
test = '/kaggle/input/multi-label-classification-competition-2024/COMP5329S1A2Dataset/test.csv'
image_dir = '/kaggle/input/multi-label-classification-competition-2024/COMP5329S1A2Dataset/data'

In [17]:
import re
from io import StringIO

In [18]:
with open(train) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
train_df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

In [19]:
with open(test) as file:
    lines = [re.sub(r'([^,])"(\s*[^\n])', r'\1/"\2', line) for line in file]
test_df = pd.read_csv(StringIO(''.join(lines)), escapechar="/")

In [7]:
train_df.head(20)

,ImageID,Labels,Caption
0,0.jpg,1,Woman in swim suit holding parasol on sunny day.
1,1.jpg,1 19,A couple of men riding horses on top of a gree...
2,2.jpg,1,They are brave for riding in the jungle on tho...
3,3.jpg,8 3 13,a black and silver clock tower at an intersect...
4,4.jpg,8 3 7,A train coming to a stop on the tracks out side.
5,5.jpg,1,A young man riding a skateboard into the air.
6,6.jpg,5,A big airplane flying in the big blue sky
7,7.jpg,1 4,A man riding a motor bike across a forest.
8,8.jpg,8 3,There is a street lined with packed buildings
9,9.jpg,1 18 15,A skate park next to a body of water and green...


In [8]:
test_df.head()

,ImageID,Caption
0,30000.jpg,A little girl waring a krispy kreme hat holdin...
1,30001.jpg,A beautiful young woman holding an orange fris...
2,30002.jpg,A group of people sitting on couch next to a c...
3,30003.jpg,A person on a snowboard rides on the hill.
4,30004.jpg,A man riding a skateboard with a helmet on in ...


In [9]:
len(train_df['Labels'].unique())

776

In [10]:
all_labels = train_df['Labels'].apply(lambda x: x.split()).explode().unique()

# Checking how many unique labels exist (should be between 1 and 19, excluding 12)
len(all_labels)

18

In [11]:

all_possible_labels = set(map(str, range(1, 20)))
missing_labels = all_possible_labels - set(all_labels)

# Output the missing label
missing_labels

{'12'}

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
 # converting all the labels into a binary matrix

train_df['Labels'] = train_df['Labels'].apply(lambda x: list(map(int, x.split())))

# all classes (e.g., excluding class 12)
all_classes = [i for i in range(1, 20) if i != 12]

# binary matrix
mlb = MultiLabelBinarizer(classes=all_classes)
binary_labels = mlb.fit_transform(train_df['Labels'])

np.save('train_labels.npy', binary_labels)
print(f"Binary labels saved: {binary_labels.shape}")

Binary labels saved: (29996, 18)


In [13]:
loaded_labels = np.load('train_labels.npy')
print(loaded_labels)
print(loaded_labels.shape)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(29996, 18)


In [14]:
sample_index = 0  # checking for image 1 
print("Original Labels:", train_df['Labels'][sample_index])
print("Binary Representation:", binary_labels[sample_index])


Original Labels: [1]
Binary Representation: [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
# Pre-processing the images

# Defines a preprocessing pipeline to resize images to 224x224, 
#convert them to tensors, and normalize pixel values using standard ImageNet mean and standard deviation.
# Reads and processes each image from the image_dir using the pipeline
# Stacks all processed image tensors into a single tensor
# Save the preprocessed images as a file
# Prints progress after every 1,000 images for monitoring.

# from torchvision import transforms
# from PIL import Image
# import torch
# import os

# # preprocessing pipeline
# image_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# # Preprocessing the training images
# def preprocess_images(image_dir, filenames, output_file):
#     image_data = []
#     for idx, img_name in enumerate(filenames):
#         img_path = os.path.join(image_dir, img_name)
#         img_tensor = image_transform(Image.open(img_path).convert('RGB'))
#         image_data.append(img_tensor)

#         if idx % 1000 == 0:
#             print(f"Processed {idx}/{len(filenames)} images...")
    
#     image_data = torch.stack(image_data)
#     torch.save(image_data, output_file)
#     print(f"Images saved to {output_file}")

# train_filenames = train_df['ImageID'].tolist()
# preprocess_images(image_dir, train_filenames, 'preprocessed_train_images.pt')


Processed 0/29996 images...
Processed 1000/29996 images...
Processed 2000/29996 images...
Processed 3000/29996 images...
Processed 4000/29996 images...
Processed 5000/29996 images...
Processed 6000/29996 images...
Processed 7000/29996 images...
Processed 8000/29996 images...
Processed 9000/29996 images...
Processed 10000/29996 images...
Processed 11000/29996 images...
Processed 12000/29996 images...
Processed 13000/29996 images...
Processed 14000/29996 images...
Processed 15000/29996 images...
Processed 16000/29996 images...
Processed 17000/29996 images...
Processed 18000/29996 images...
Processed 19000/29996 images...
Processed 20000/29996 images...
Processed 21000/29996 images...
Processed 22000/29996 images...
Processed 23000/29996 images...
Processed 24000/29996 images...
Processed 25000/29996 images...
Processed 26000/29996 images...
Processed 27000/29996 images...
Processed 28000/29996 images...
Processed 29000/29996 images...


In [21]:
# # Preprocessinf the  test images
# test_filenames = test_df['ImageID'].tolist()
# preprocess_images(image_dir, test_filenames, 'preprocessed_test_images.pt')

In [23]:
import os
import torch
from torchvision import transforms
from PIL import Image
# processing all images at once caused the kernel to restart again and again
# so doing it in chunks of 5000 images at a time.

# preprocessing pipeline
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

def preprocess_images_in_chunk(image_dir, filenames, chunk_size, output_dir):
    """
    Preprocess images in smaller chunks and save each chunk separately.
    Args:
        image_dir: Path to the folder containing images.
        filenames: List of image filenames to process.
        chunk_size: Number of images to process in one chunk.
        output_dir: Directory to save the preprocessed chunks.
    """
    os.makedirs(output_dir, exist_ok=True)
    total_files = len(filenames)
    
    for start_idx in range(0, total_files, chunk_size):
        end_idx = min(start_idx + chunk_size, total_files)
        chunk_filenames = filenames[start_idx:end_idx]
        
        image_data = []
        for img_name in chunk_filenames:
            img_path = os.path.join(image_dir, img_name)
            img_tensor = image_transform(Image.open(img_path).convert('RGB'))
            image_data.append(img_tensor)
        
        # Save the current chunk
        chunk_file = os.path.join(output_dir, f'preprocessed_images_{start_idx}_{end_idx}.pt')
        torch.save(torch.stack(image_data), chunk_file)
        print(f"Saved chunk {start_idx}-{end_idx} to {chunk_file}")


train_filenames = train_df['ImageID'].tolist()
chunk_size = 5000  # Process 5,000 images at a time
output_dir = './preprocessed_train_chunks'

preprocess_images_in_chunks(image_dir, train_filenames, chunk_size, output_dir)


Saved chunk 0-5000 to ./preprocessed_train_chunks/preprocessed_images_0_5000.pt
Saved chunk 5000-10000 to ./preprocessed_train_chunks/preprocessed_images_5000_10000.pt
Saved chunk 10000-15000 to ./preprocessed_train_chunks/preprocessed_images_10000_15000.pt
Saved chunk 15000-20000 to ./preprocessed_train_chunks/preprocessed_images_15000_20000.pt
Saved chunk 20000-25000 to ./preprocessed_train_chunks/preprocessed_images_20000_25000.pt
Saved chunk 25000-29996 to ./preprocessed_train_chunks/preprocessed_images_25000_29996.pt
